In [2]:
import mlflow
import pandas as pd
import time
import mlflow.system_metrics
from logging import getLogger, StreamHandler, INFO
import xgboost
import shap
import mlflow
from sklearn.model_selection import train_test_split

/workspace/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mlflow.set_tracking_uri(
    "http://ec2-44-217-145-52.compute-1.amazonaws.com:5000"
)
experiment = mlflow.set_experiment("test")
mlflow.system_metrics.enable_system_metrics_logging()

In [4]:
run = mlflow.start_run(experiment_id=experiment.experiment_id)

2024/10/26 09:17:28 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


In [5]:
mlflow.log_metrics({"tlearner_auuc": 0.1, "tlearner_uplift": 0.01})
time.sleep(10)
mlflow.log_metrics({"slearner_auuc": 0.2, "tlearner_uplift": 0.02})

In [6]:
# load UCI Adult Data Set; segment it into training and test sets
X, y = shap.datasets.adult()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# train XGBoost model
model = xgboost.XGBClassifier().fit(X_train, y_train)

# construct an evaluation dataset from the test set
eval_data = X_test
eval_data["target"] = y_test

In [7]:
# load UCI Adult Data Set; segment it into training and test sets
X, y = shap.datasets.adult()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# train XGBoost model
model = xgboost.XGBClassifier().fit(X_train, y_train)

# construct an evaluation dataset from the test set
eval_data = X_test
eval_data["target"] = y_test

In [1]:

model_info = mlflow.sklearn.log_model(model, "model")
result = mlflow.evaluate(
    model_info.model_uri,
    eval_data,
    targets="target",
    model_type="classifier",
    evaluators=["default"],
)

NameError: name 'mlflow' is not defined

In [24]:
mlflow.log_table(eval_data, artifact_file="eval_data.json")

In [8]:
mlflow.end_run()

2024/10/26 09:24:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run handsome-midge-696 at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/1/runs/0f89f83ef08b45c2b0e0f8868a02f351.
2024/10/26 09:24:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/1.
2024/10/26 09:24:12 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/10/26 09:24:12 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
